In [31]:
import pandas as pd
import pprint
import os
import numpy as np
import time
import re
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.autograd import Variable
import torchvision.datasets as vdatasets
import torchvision.transforms as transforms
from torchvision.utils import save_image

In [2]:
df1 = pd.read_csv('lotto.csv'
                   , engine='python'                   
                   , names = ['1','2','3','4','5','6','7']
                   #, encoding='utf-8'                   
                   #, na_filter =True                   
                  )
df1=df1.drop(['7'],axis=1)
df1



,1,2,3,4,5,6
0,1,3,12,14,16,43
1,8,11,19,21,36,45
2,5,10,13,21,39,43
3,6,11,15,17,23,40
4,15,21,31,32,41,43
5,6,10,18,25,34,35
6,14,20,23,31,37,38
7,3,12,13,18,31,32
8,1,10,13,26,32,36
9,5,9,14,26,30,43


In [40]:
lotto_data = []
for st in range(110):
    emp=[]
    for idx in range(6):
        idx = st+idx
        emp.append(df1.loc[idx].tolist())
    lotto_data.append(emp)

In [47]:
lotto_data=torch.Tensor(lotto_data)
lotto_data

tensor([[[  1.,   3.,  12.,  14.,  16.,  43.],
         [  8.,  11.,  19.,  21.,  36.,  45.],
         [  5.,  10.,  13.,  21.,  39.,  43.],
         [  6.,  11.,  15.,  17.,  23.,  40.],
         [ 15.,  21.,  31.,  32.,  41.,  43.],
         [  6.,  10.,  18.,  25.,  34.,  35.]],

        [[  8.,  11.,  19.,  21.,  36.,  45.],
         [  5.,  10.,  13.,  21.,  39.,  43.],
         [  6.,  11.,  15.,  17.,  23.,  40.],
         [ 15.,  21.,  31.,  32.,  41.,  43.],
         [  6.,  10.,  18.,  25.,  34.,  35.],
         [ 14.,  20.,  23.,  31.,  37.,  38.]],

        [[  5.,  10.,  13.,  21.,  39.,  43.],
         [  6.,  11.,  15.,  17.,  23.,  40.],
         [ 15.,  21.,  31.,  32.,  41.,  43.],
         [  6.,  10.,  18.,  25.,  34.,  35.],
         [ 14.,  20.,  23.,  31.,  37.,  38.],
         [  3.,  12.,  13.,  18.,  31.,  32.]],

        ...,

        [[  1.,   6.,  17.,  22.,  28.,  45.],
         [  1.,   4.,   8.,  23.,  33.,  42.],
         [ 10.,  28.,  31.,  33.,  41., 

In [43]:
sum((lotto_data[0]),[])

[1,
 3,
 12,
 14,
 16,
 43,
 8,
 11,
 19,
 21,
 36,
 45,
 5,
 10,
 13,
 21,
 39,
 43,
 6,
 11,
 15,
 17,
 23,
 40,
 15,
 21,
 31,
 32,
 41,
 43,
 6,
 10,
 18,
 25,
 34,
 35]

In [52]:
# Hyper Parameters 
INPUT_SIZE = 36
HIDDEN_SIZE = 128
LATENT_SIZE = 6
EPOCH = 10000
BATCH_SIZE = 1

In [27]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

In [29]:
Generator = nn.Sequential(nn.Linear(LATENT_SIZE,HIDDEN_SIZE),
                                        nn.LeakyReLU(0.2),
                                        nn.Linear(HIDDEN_SIZE,HIDDEN_SIZE),
                                        nn.LeakyReLU(0.2),
                                        nn.Linear(HIDDEN_SIZE,INPUT_SIZE),
                                        nn.Tanh())

Discriminator = nn.Sequential(nn.Linear(INPUT_SIZE,HIDDEN_SIZE),
                                             nn.LeakyReLU(0.2),
                                             nn.Linear(HIDDEN_SIZE,HIDDEN_SIZE),
                                             nn.LeakyReLU(0.2),
                                             nn.Linear(HIDDEN_SIZE,1),
                                             nn.Sigmoid())

In [32]:
LR = 0.0002

loss_function = nn.BCELoss()  
d_optimizer = optim.Adam(Discriminator.parameters(), lr=LR)
g_optimizer = optim.Adam(Generator.parameters(),lr=LR)

In [53]:
for epoch in range(EPOCH):
    for i, inputs in enumerate(lotto_data):
        
        # Generator 학습
        real_img = Variable(inputs.view(-1,6*6))
        real_label = Variable(torch.ones(1))
        real_preds = Discriminator(real_img)
        
        latent = Variable(torch.randn(1,LATENT_SIZE))
        fake_img = Generator(latent)
        fake_label = Variable(torch.zeros(1))
        fake_preds = Discriminator(fake_img)
        
        d_loss_1 = loss_function(real_preds.squeeze(1),real_label)
        d_loss_2 = loss_function(fake_preds.squeeze(1),fake_label)
        
        d_loss = d_loss_1 + d_loss_2
        
        Discriminator.zero_grad()
        Generator.zero_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # Discriminator 학습
        latent = Variable(torch.randn(1,LATENT_SIZE))
        fake_img = Generator(latent)
        preds = Discriminator(fake_img)
        
        g_loss = loss_function(preds.squeeze(1),real_label)
        
        Discriminator.zero_grad()
        Generator.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 300 == 0:
            print ('Epoch: [%d/%d], Step: [%d/%d], D Loss: %.4f, G Loss: %.4f' 
                   % (epoch+1, EPOCH, i+1, len(train_dataset)//BATCH_SIZE, d_loss.data[0],g_loss.data[0]))
            
    # 생성한 이미지 샘플 저장
    if (epoch+1) % 10 == 0:
        fake_img = fake_img.view(1, 1, 6, 6)
        save_image(denorm(fake_img.data), './images/gan_gen_images-%d.png' %(epoch+1))